<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex13notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex13notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## 演習問題 - 猫顔画像および手書き数字画像のクラスタリング
----

次の二種類のデータでクラスタリングの実験をやってみましょう．

- 猫の顔の画像131枚
- 手書き数字の画像 5000 枚




In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set_theme()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

---
### 猫顔画像のクラスタリング

#### 準備

画像を並べて表示する関数を定義しておきます．

In [ ]:
#####  データの最初の nx x ny 枚を可視化
#
def mosaicImage(ax, dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype=int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))

    # 表示
    ax.axis('off')
    ax.imshow(img, cmap='gray')

猫顔画像データを入手して一部を表示してみます．

In [ ]:
# 猫顔画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/cat131.npz
X_cat = np.load('cat131.npz')['cat131']
N_cat, D_cat = X_cat.shape
print(f'データ数 x 次元数 = {N_cat} x {D_cat}')

# 最初の 20 枚を表示
fig, ax = plt.subplots(figsize=(6, 6))
mosaicImage(ax, X_cat, 5, 4)
plt.show()

このデータは，猫の顔の画像 131 枚から成ります．1枚の画像は $64\times 64$ 画素のグレイスケール画像です．上のコードセルを実行すると，`X_cat` という変数にデータが格納されます．

#### 階層型クラスタリングしてみよう



上記の配列 `X_cat` に格納されたデータを対象に階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
link = hierarchy.linkage(X_cat, method='ward', metric='euclidean')
fig, ax = plt.subplots(figsize=(8, 20))
hierarchy.dendrogram(link, orientation='right', distance_sort='descending', show_leaf_counts=True, leaf_font_size=8, ax=ax)
plt.show()

次のセルを実行すると，`threshold` で指定した値をクラスタ間距離のしきい値として画像をクラスタに分け，それぞれのクラスタに属する画像の数を表示します．

In [ ]:
threshold = 20000 # クラスタ間距離のしきい値
label = hierarchy.fcluster(link, t=threshold, criterion='distance')
K = max(label) # クラスタ数
print(f'threshold = {threshold}, K = {K}')
for ik in range(K):
    print(f'cluster{ik+1}: num = {np.sum(label == ik+1)}')

次のセルを実行すると，上で得られたクラスタリングの結果を画像として表示します．
一つのクラスタに属する画像が横に並んでいます（横一行が同じクラスタに属する画像たちの集まり）．

In [ ]:
# 各クラスタに属する画像を表示

nc = 10  # 1行に並べる画像の最大数

dat = np.zeros((K, nc, D_cat))
for ik in range(K):
    idx = label == (ik+1)
    n = np.sum(idx)
    if n < nc:
        dat[ik, :n, :] = X_cat[idx, :]
    else:
        dat[ik, :, :] = X_cat[idx, :][:nc, :]

dat = dat.reshape((-1, D_cat))

fig, ax = plt.subplots(figsize=(10, 10))
mosaicImage(ax, dat, nc, K)
plt.show()

**問題1**

次のことを考えて／調べて書き留めておこう：

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


**問題2**

次のことをやろう：

(1) クラスタ間距離のしきい値をいろいろ変えて結果を観察しよう．

(2) クラスタ間距離のしきい値が 6000 のときのクラスタ数と，各クラスタに属する画像の数の最大と最小を書き留めておこう．

(3) クラスタ数が 13 になるようなクラスタ間距離のしきい値を求め，そのときに各クラスタに属する画像の数の最大と最小を書き留めておこう．

---
### 手書き数字画像のクラスタリング

判別分析の回でも用いた手書き数字の画像でクラスタリングの実験をやってみましょう．

#### 準備


In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
X_digits = rv['datL'].astype(float)
N_digits, D_digits = X_digits.shape

In [ ]:
print(f'データ数 x 次元数 = {X_digits.shape[0]} x {X_digits.shape[1]}')
# 最初の 60 枚を表示
fig, ax = plt.subplots(figsize=(10, 6))
mosaicImage(ax, X_digits, 15, 4, nrow=28, ncol=28)
plt.show()

このデータは，上図のような 0 から 9 までの手書き数字画像 5000 枚から成ります．1枚の画像は $28\times 28$ 画素のグレイスケール画像です．上のコードセルを実行すると，`X_digits` という変数にデータが格納されます．

このデータセットは，機械学習の分野で例題としてよく用いられる MNIST（[Wikipediaの記事](https://ja.wikipedia.org/wiki/MNIST%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9)） という手書き数字画像のデータセットから一部を抽出したものになっています．

#### 階層型クラスタリングしてみよう

上記の配列 `X_digits` に格納されたデータを対象に階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
link = hierarchy.linkage(X_digits, method='ward', metric='euclidean')
fig, ax = plt.subplots(figsize=(10, 8))
hierarchy.dendrogram(link, ax=ax, truncate_mode='lastp', p=50, distance_sort='descending')
plt.show()

この実験ではデータ数が非常に多く，完全なデンドログラムを描くのは大変です．そのため，上記の図では，クラスター数が50以上になる枝分かれは省略して描いてあります．実際のデンドログラムは，一つのクラスタに一つのデータしか入らないところまで枝分かれしていることに注意．

次の二つのセルを実行すると，猫顔画像の実験と同様にクラスタリング結果を出力させることができます．

In [ ]:
threshold = 30000 # クラスタ間距離のしきい値
label = hierarchy.fcluster(link, t=threshold, criterion='distance')
K = max(label) # クラスタ数
print(f'threshold = {threshold}, K = {K}')
for ik in range(K):
    print(f'cluster{ik+1}: num = {np.sum(label == ik+1)}')

In [ ]:
# 各クラスタに属する画像を表示
#    1行が1クラスタ．1行あたり最大 nc 枚を表示
nc = 16
dat = np.zeros((K, nc, D_digits))
for ik in range(K):
    idx = label == (ik+1)
    n = np.sum(idx)
    if n < nc:
        dat[ik, :n, :] = X_digits[idx, :]
    else:
        dat[ik, :, :] = X_digits[idx, :][:nc, :]

dat = dat.reshape((-1, D_digits))

fig, ax = plt.subplots(figsize=(10, 12))
mosaicImage(ax, dat, nc, K, nrow=28, ncol=28)
plt.show()

**問題3**

次のことを考えて／調べてメモしておこう：

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


**問題4**

次のことをやろう：

(1) クラスタ間距離のしきい値をいろいろ変えて結果を観察しよう．

(2) クラスタ間距離のしきい値が 18800 のときのクラスタ数を書き留めておこう．